In [4]:
!pip3 install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/lts/stable

Looking in links: https://download.pytorch.org/whl/cu111/lts/stable
  Using cached torch-1.10.2-cp36-cp36m-manylinux2014_aarch64.whl (53.5 MB)
  Using cached torchvision-0.11.3-cp36-cp36m-manylinux2014_aarch64.whl (14.7 MB)
     |################################| 2.7 MB 4.7 MB/s            


In [ ]:
#import cv2
#import numpy as np
#from rplidar import RPLidar
import torch
#import torchvision.transforms as T
#from torchvision.models import resnet50
#from IPython.display import clear_output, display
#import ipywidgets as widgets
#from IPython.display import HTML


In [ ]:
# Constants
MIN_DISTANCE = 1000  # Minimum distance for object avoidance in mm

# Initialize RPLidar
lidar = RPLidar('/dev/ttyUSB0')  # Update with your RPLidar's port

# Initialize webcam
cap = cv2.VideoCapture(0)

# Load pre-trained object detection model
model = resnet50(pretrained=True)
model.eval()

# Function to get object detection results
def detect_objects(image, model):
    transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor(), T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    input_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)
    return output

# Function to process LIDAR data for object avoidance
def process_lidar_data(lidar_data):
    for angle, distance in lidar_data:
        if distance < MIN_DISTANCE:
            return True  # Object detected within the minimum distance
    return False

# Function to display LIDAR data as a simple plot
def display_lidar_data(lidar_data):
    import matplotlib.pyplot as plt

    angle = [item[1] for item in lidar_data]
    distance = [item[2] for item in lidar_data]

    plt.figure(figsize=(8, 8))
    plt.scatter(angle, distance)
    plt.xlabel('Angle [deg]')
    plt.ylabel('Distance [mm]')
    plt.title('LIDAR Data')
    plt.grid()
    plt.show()

# Function to show an image in Jupyter Notebook
def show_image(image):
    _, encoded_image = cv2.imencode('.jpg', image)
    display(HTML(data=f'<img src="data:image/jpg;base64,{encoded_image.tobytes().decode("utf-8")}" alt="Webcam Image"/>'))


In [ ]:
try:
    while True:
        # Read image from webcam
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object identification
        output = detect_objects(frame, model)
        # You may want to process the output further to extract the detected objects and their labels

        # Get LIDAR data
        lidar_data = [item for item in lidar.iter_scans()]

        # Perform object avoidance
        if process_lidar_data(lidar_data):
            print("Obstacle detected! Take necessary action.")
            # Insert your control code here to take necessary action when an obstacle is detected

        # Display the webcam frame
        clear_output(wait=True)
        show_image(frame)

        # Display the LIDAR data
        display_lidar_data(lidar_data)

except KeyboardInterrupt:
    print("Stopping...")
finally:
    cap.release()
    lidar.stop()
    lidar.disconnect()


In [ ]:
import os

# Create output directory if it doesn't exist
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Initialize VideoWriter for raw video
raw_video_out = cv2.VideoWriter(os.path.join(output_dir, 'raw_video.avi'), cv2.VideoWriter_fourcc(*'XVID'), 20, (int(cap.get(3)), int(cap.get(4))))

# Initialize VideoWriter for detection video
detection_video_out = cv2.VideoWriter(os.path.join(output_dir, 'detection_video.avi'), cv2.VideoWriter_fourcc(*'XVID'), 20, (int(cap.get(3)), int(cap.get(4))))

# Function to draw LIDAR data overlay on the frame
def draw_lidar_overlay(frame, lidar_data):
    overlay_frame = frame.copy()
    for angle, distance in lidar_data:
        x = int(frame.shape[1]/2 + distance * np.cos(np.radians(angle)))
        y = int(frame.shape[0]/2 - distance * np.sin(np.radians(angle)))
        cv2.circle(overlay_frame, (x, y), 2, (0, 255, 0), -1)
    return overlay_frame

try:
    while True:
        # Read image from webcam
        ret, frame = cap.read()
        if not ret:
            break

        # Perform object identification
        output = detect_objects(frame, model)
        # You may want to process the output further to extract the detected objects and their labels

        # Get LIDAR data
        lidar_data = [item for item in lidar.iter_scans()]

        # Perform object avoidance
        if process_lidar_data(lidar_data):
            print("Obstacle detected! Take necessary action.")
            # Insert your control code here to take necessary action when an obstacle is detected

        # Save raw video frame
        raw_video_out.write(frame)

        # Draw LIDAR overlay on the frame
        lidar_overlay_frame = draw_lidar_overlay(frame, lidar_data)

        # Save detection video frame
        detection_video_out.write(lidar_overlay_frame)

except KeyboardInterrupt:
    print("Stopping...")
finally:
    cap.release()
    raw_video_out.release()
    detection_video_out.release()
    lidar.stop()
    lidar.disconnect()
